## Preposcessing with ES & EZ

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
from datetime import date
import pickle

In [2]:
# 1202 in es while 1201 in ez
# Read in the data and combine them into one dataframe
es_ez_date = np.array([
    20200102, 20200115, 20200203, 20200214, 20200302, 20200316, 20200401, 20200415, 20200501, 20200515, 20200601,
    20200615, 20200701, 20200715, 20200803, 20200814, 20200901, 20200915, 20201001, 20201015, 20201102, 20201116,
    20201215, 20210104, 20210115, 20210201, 20210216, 20210301, 20210315 
])
ES_EZ = None
for i in es_ez_date:
    # Read the es data
    es_path = '/Users/gaojinglun/Desktop/eod/es/' + str(i) + '.csv'
    ES_small = pd.read_csv(es_path, usecols = ['Trade.Date', 'Contract.Year', 
                                               'Contract.Month', 'Settlement', 'Last.Trade.Date'])
    
    # Read the ez data
    ez_path = '/Users/gaojinglun/Desktop/eod/ez/' + str(i) + '.csv'
    EZ_small = pd.read_csv(ez_path, usecols = ['Trade.Date', 'Put.Call', 'Strike.Price', 'Contract.Year', 
                                               'Contract.Month', 'Settlement', 'Open.Interest',
                                               'Delta', 'Implied.Volatility', 'Last.Trade.Date'])
    
    # Add the futures price
    futures_price = np.zeros(EZ_small.shape[0])
    for i in range(ES_small.shape[0]):
        ContractYear = ES_small['Contract.Year'].values[i]
        ContractMonth = ES_small['Contract.Month'].values[i]
        same_year_month = np.logical_and(EZ_small['Contract.Year'] == ContractYear,
                                         EZ_small['Contract.Month'] == ContractMonth)
        futures_price[same_year_month] = ES_small['Settlement'][i]                                 
            
    assert np.sum(futures_price == 0) == 0, 'futures price should be positive!'
    
    EZ_small['futures.price'] = futures_price
    ES_EZ = pd.concat([ES_EZ, EZ_small], axis = 0)
    
    del ES_small    
    del EZ_small

In [3]:
print('There are {} rows and {} columns in ES_EZ data'.format(ES_EZ.shape[0], ES_EZ.shape[1]))

There are 29971 rows and 11 columns in ES_EZ data


In [4]:
ES_EZ.head()

,Trade.Date,Put.Call,Strike.Price,Contract.Year,Contract.Month,Settlement,Open.Interest,Delta,Implied.Volatility,Last.Trade.Date,futures.price
0,20200102,C,2200,2020,3,1059.0,826.0,0.99787,0.338267,20200320,3259.0
1,20200102,C,2250,2020,3,1009.1,728.0,0.99661,0.326829,20200320,3259.0
2,20200102,C,2270,2020,3,989.1,1.0,0.99586,0.323456,20200320,3259.0
3,20200102,C,2300,2020,3,959.2,212.0,0.99474,0.317834,20200320,3259.0
4,20200102,C,2310,2020,3,949.2,42.0,0.99466,0.314282,20200320,3259.0


In [5]:
# Calculate the time to maturity
T = np.zeros(ES_EZ.shape[0])
for i in range(len(T)):
    dateInString = str(ES_EZ['Trade.Date'].values[i])
    start_year = int(dateInString[:4])
    
    if dateInString[4] == str(0):
        start_month = int(dateInString[5])
    else:
        start_month = int(dateInString[4:6])
    
    if dateInString[6] == str(0):
        start_day = int(dateInString[-1])
    else:
        start_day = int(dateInString[6:])
    
    # Save the start date in date
    start_date = date(start_year, start_month, start_day)
    
    endDateInString = str(ES_EZ['Last.Trade.Date'].values[i])
    end_year = int(endDateInString[:4])
    
    if endDateInString[4] == str(0):
        end_month = int(endDateInString[5])
    else:
        end_month = int(endDateInString[4:6])
    
    if endDateInString[6] == str(0):
        end_day = int(endDateInString[-1])
    else:
        end_day = int(endDateInString[6:])
    
    # Save the start date in date
    end_date = date(end_year, end_month, end_day)
    
    T[i] = (end_date - start_date).days
    
assert np.sum(T == 0) == 0, 'Time should be positive!'

In [6]:
ES_EZ['Time.to.maturity'] = T / 365
ES_EZ.head()

,Trade.Date,Put.Call,Strike.Price,Contract.Year,Contract.Month,Settlement,Open.Interest,Delta,Implied.Volatility,Last.Trade.Date,futures.price,Time.to.maturity
0,20200102,C,2200,2020,3,1059.0,826.0,0.99787,0.338267,20200320,3259.0,0.213699
1,20200102,C,2250,2020,3,1009.1,728.0,0.99661,0.326829,20200320,3259.0,0.213699
2,20200102,C,2270,2020,3,989.1,1.0,0.99586,0.323456,20200320,3259.0,0.213699
3,20200102,C,2300,2020,3,959.2,212.0,0.99474,0.317834,20200320,3259.0,0.213699
4,20200102,C,2310,2020,3,949.2,42.0,0.99466,0.314282,20200320,3259.0,0.213699


In [7]:
# Add the risk-free-rate
# T-bill
T_bill_rate = np.zeros(len(T))

def t_bill_distance(x):
    '''
    Find the similar type of the T-bill based on the time to maturity
    '''
    one_month_t_bill = int(365 / 12)
    three_month_t_bill = int(365 / 4)
    six_month_t_bill = int(365 / 2)
    one_year_t_bill = int(365 / 1)
    idx = np.argmin([
        np.abs(x - one_month_t_bill), np.abs(x - three_month_t_bill), 
        np.abs(x - six_month_t_bill), np.abs(x - one_year_t_bill)
    ])
    return [0, 1, 2, 3][idx]

for i in range(len(T)):
    T_bill_rate[i] = t_bill_distance(T[i])
np.unique(T_bill_rate)

array([0., 1., 2., 3.])

In [8]:
TB_data = np.array(['DGS1MO', 'DGS3MO', 'DGS6MO', 'DGS1'])
TB = []
TB_date_new = []
for i in range(4):
    TB_path = '/Users/gaojinglun/Desktop/eod/' + TB_data[i] + '.csv'
    TB_small = pd.read_csv(TB_path)
    
    def DATE_convertor(x):
        '''
        convert xxxx-xx-xx to xxxxxxxx
        '''
        return x[:4] + x[5:7] + x[-2:]

    date_idx = np.zeros(TB_small.shape[0], dtype = bool)
    for i in range(TB_small.shape[0]):
        new_date = int(DATE_convertor(TB_small['DATE'].values[i]))
        if new_date in es_ez_date:
            date_idx[i] = True
            TB_date_new.append(new_date)
    
    TB.append(TB_small[TB_small.columns.values[-1]][date_idx].values)
    del TB_small
    
TB = np.array(TB)
TB_date_new = TB_date_new[:int(len(TB_date_new) / 4)]
TB.shape

(4, 29)

In [9]:
risk_free_rate = np.zeros(ES_EZ.shape[0])
for i in range(ES_EZ.shape[0]):
    risk_free_rate[i] = TB[int(T_bill_rate[i]), TB_date_new.index(ES_EZ['Trade.Date'].values[i])]

In [10]:
ES_EZ['Risk.Free.Rate'] = risk_free_rate / 100
ES_EZ.head()

,Trade.Date,Put.Call,Strike.Price,Contract.Year,Contract.Month,Settlement,Open.Interest,Delta,Implied.Volatility,Last.Trade.Date,futures.price,Time.to.maturity,Risk.Free.Rate
0,20200102,C,2200,2020,3,1059.0,826.0,0.99787,0.338267,20200320,3259.0,0.213699,0.0154
1,20200102,C,2250,2020,3,1009.1,728.0,0.99661,0.326829,20200320,3259.0,0.213699,0.0154
2,20200102,C,2270,2020,3,989.1,1.0,0.99586,0.323456,20200320,3259.0,0.213699,0.0154
3,20200102,C,2300,2020,3,959.2,212.0,0.99474,0.317834,20200320,3259.0,0.213699,0.0154
4,20200102,C,2310,2020,3,949.2,42.0,0.99466,0.314282,20200320,3259.0,0.213699,0.0154


In [11]:
ES_EZ.to_csv("/Users/gaojinglun/Desktop/RSG/data/ES_EZ.csv")

**Reference**

- Board of Governors of the Federal Reserve System (US), Market Yield on U.S. Treasury Securities at 1-Month Constant Maturity [DGS1MO], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/DGS1MO, November 7, 2021.

- Board of Governors of the Federal Reserve System (US), Market Yield on U.S. Treasury Securities at 3-Month Constant Maturity [DGS3MO], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/DGS3MO, November 7, 2021.

- Board of Governors of the Federal Reserve System (US), Market Yield on U.S. Treasury Securities at 6-Month Constant Maturity [DGS6MO], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/DGS6MO, November 7, 2021.

- Board of Governors of the Federal Reserve System (US), Market Yield on U.S. Treasury Securities at 1-Year Constant Maturity [DGS1], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/DGS1, November 7, 2021.